<a href="https://colab.research.google.com/github/stuart-lane/MachineLearning/blob/main/LogitSimulations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [234]:
import numpy as np
import pandas as pd
import random

np.random.seed(123456)
#np.random.seed(7542716)

DATA GENERATING PROCESS

In [257]:
n = 1000
x = np.random.randn(n)
u = np.random.logistic(0, 1, n)
beta0 = 0
beta1 = 1

ystar = beta0 + beta1 * x + u
y = (ystar > 0).astype(int)

#p = 1 / (1 + np.exp(-(beta0 + x*beta1 + u)))
#y = np.random.binomial(n = 1, p = p, size = n)

In [258]:
from sklearn.model_selection import train_test_split

u = u.reshape(-1, 1)
x = np.hstack((np.ones((n, 1)), x.reshape(-1, 1)))
y = y.reshape(-1, 1)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 123)

LOGISTIC REGRESSION WITH SKLEARN

In [259]:
from sklearn.linear_model import LogisticRegression

In [260]:
model = LogisticRegression(solver = 'lbfgs', max_iter = 1000)  # You can change the solver and max_iter as needed
model.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(max_iter=1000)

In [261]:
intercept = model.intercept_[0]
coefficient = model.coef_[0][1]

In [262]:
print(f"Intercept: {intercept}")
print(f"Coefficient for 'Regressor': {coefficient}")

Intercept: -0.060131908967700765
Coefficient for 'Regressor': 1.253262836265797


STANDARD LOGISTIC REGRESSION

In [241]:
# Define the logistic function
def logistic(x, beta):
    x_beta = np.dot(x, beta)
    x_beta = np.clip(x_beta, -500, 500)
    return 1 / (1 + np.exp(-x_beta))

# Define the log-likelihood function
def log_likelihood(beta, x, y):
    p = logistic(x, beta)
    epsilon = 1e-10
    p = np.clip(p, epsilon, 1 - epsilon)
    ll = np.sum(y * np.log(p) + (1 - y) * np.log(1 - p))
    return -ll

def derivative(beta, x_train, y_train):
    p = logistic(x_train, beta)
    gradient = np.zeros(2)  # Initialize a 1D gradient array with size 2
    gradient[0] = np.sum(y_train - p)
    gradient[1] = np.sum(np.dot(x_train.T, y_train - p)) / x_train.shape[0]
    return gradient

beta_initial = np.array([0.5, 0.5])

from scipy.optimize import minimize
result = minimize(log_likelihood, beta_initial, args=(x_train, y_train), method = 'L-BFGS-B', jac = derivative)

print(result)

estimated_beta = [f'{param:.7f}' for param in result.x]
formatted_parameters = ', '.join(estimated_beta)

print(f"Estimated Parameters: [{formatted_parameters}]")
print("Negative Log-Likelihood:", result.fun)

  message: ABNORMAL_TERMINATION_IN_LNSRCH
  success: False
   status: 2
      fun: 491889.4423803657
        x: [ 5.000e-01  5.000e-01]
      nit: 0
      jac: [-9.307e+04  5.957e+01]
     nfev: 20
     njev: 20
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
Estimated Parameters: [0.5000000, 0.5000000]
Negative Log-Likelihood: 491889.4423803657


In [266]:
import numpy as np
from scipy.optimize import minimize

# Define the logistic function
def logistic(x, beta):
    x_beta = np.dot(x, beta)
    x_beta = np.clip(x_beta, -500, 500)
    return 1 / (1 + np.exp(-x_beta))

# Define the log-likelihood function
def log_likelihood(beta, x, y):
    p = logistic(x, beta)
    epsilon = 1e-10
    p = np.clip(p, epsilon, 1 - epsilon)
    ll = np.sum(y * np.log(p) + (1 - y) * np.log(1 - p))
    return -ll

# Define the gradient (Jacobian) of the log-likelihood function
def gradient(beta, x, y):
    p = logistic(x, beta)
    gradient_vector = np.dot(x.T, (y - p)) / x.shape[0]
    return -gradient_vector

beta_initial = np.array([0, 1])

# Example usage of minimize
result = minimize(log_likelihood, beta_initial, args=(x_train, y_train), method='BFGS')

print(result)

estimated_beta = [f'{param:.7f}' for param in result.x]
formatted_parameters = ', '.join(estimated_beta)

print(f"Estimated Parameters: [{formatted_parameters}]")
print("Negative Log-Likelihood:", result.fun)

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 443516.1905556687
        x: [-3.500e-02  1.115e-08]
      nit: 8
      jac: [ 0.000e+00  1.172e-02]
 hess_inv: [[ 3.128e-06  2.009e-06]
            [ 2.009e-06  2.201e-06]]
     nfev: 36
     njev: 12
Estimated Parameters: [-0.0350036, 0.0000000]
Negative Log-Likelihood: 443516.1905556687


In [255]:
# Define the logistic function
#def logistic(x, beta):
#    x_beta = np.dot(x, beta)
#    x_beta = np.clip(x_beta, -500, 500)
#    return 1 / (1 + np.exp(-x_beta))

# Define the log-likelihood function
#def log_likelihood(beta, x, y):
#    x
#    p = logistic(x, beta)
#    epsilon = 1e-10
#    p = np.clip(p, epsilon, 1 - epsilon)
#    ll = np.sum(y * np.log(p) + (1 - y) * np.log(1 - p))
#    return -ll

# Define the log-likelihood function
def log_likelihood(beta, x, y):
    x_beta = x @ beta
    p = 1 / (1 + np.exp(-x_beta))
    epsilon = 1e-10
    p = np.clip(p, epsilon, 1 - epsilon)
    ll = np.sum(y * np.log(p) + (1 - y) * np.log(1 - p))
    return -ll

beta_initial = np.array([0, 1])


from scipy.optimize import minimize
result = minimize(log_likelihood, beta_initial, args=(x_train, y_train), method = 'L-BFGS-B')

print(result)

estimated_beta = [f'{param:.7f}' for param in result.x]
formatted_parameters = ', '.join(estimated_beta)

print(f"Estimated Parameters: [{formatted_parameters}]")
print("Negative Log-Likelihood:", result.fun)

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 441930.7189702924
        x: [-1.453e-01  1.547e-05]
      nit: 4
      jac: [-3.783e-01  2.643e+00]
     nfev: 15
     njev: 5
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
Estimated Parameters: [-0.1452562, 0.0000155]
Negative Log-Likelihood: 441930.7189702924


In [243]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def logistic_regression(X, y, learning_rate, num_iterations):
    m, n = X.shape
    theta = np.zeros((n, 1))

    for i in range(num_iterations):
        z = np.dot(X, theta)
        h = sigmoid(z)
        gradient = np.dot(X.T, (h - y)) / m
        theta -= learning_rate * gradient
        if (i+1) % 100 == 0:
          print(f"Iteration: {i+1} || Gradients: {gradient[0][0]} and {gradient[1][0]}")
          print(f"Estimated Parameters (Weights): {theta[0][0]} and {theta[1][0]}")

    return theta

learning_rate = 0.1
num_iterations = 1000

theta = logistic_regression(x_train, y_train, learning_rate, num_iterations)

#print("Estimated Parameters (Weights):", theta)

Iteration: 100 || Gradients: 0.0024484652999759928 and -0.024826645218115048
Estimated Parameters (Weights): -0.11329131281618507 and 0.81057807398994
Iteration: 200 || Gradients: 0.0003652894496281411 and -0.00469650720890295
Estimated Parameters (Weights): -0.12381547601229506 and 0.9287924239136948
Iteration: 300 || Gradients: 6.741809573183612e-05 and -0.0009668692203220522
Estimated Parameters (Weights): -0.1255479423812796 and 0.9521417298211473
Iteration: 400 || Gradients: 1.3323826691407498e-05 and -0.00020236601955596283
Estimated Parameters (Weights): -0.1258777871830009 and 0.9569891455894065
Iteration: 500 || Gradients: 2.7082526901361924e-06 and -4.249818941969404e-05
Estimated Parameters (Weights): -0.12594378392045522 and 0.9580054499044263
Iteration: 600 || Gradients: 5.583272185494259e-07 and -8.93091069069759e-06
Estimated Parameters (Weights): -0.1259572801211017 and 0.9582189541023278
Iteration: 700 || Gradients: 1.1601941852526498e-07 and -1.8770430608018932e-06
Es

AUTOGRAD

In [244]:
import torch
from torch.optim import Adam
from torch.optim import LBFGS

In [ ]:
train_df_keys = {'constant': x_train[:,0].flatten(), 'X1': x_train[:,1].flatten(), 'y': y_train.flatten()}

train_df = pd.DataFrame(train_df_keys)
train_df.head(20)

In [246]:
x = torch.cat((torch.tensor(train_df[["constant", "X1"]].values, dtype=torch.float64),), dim=1)
x.requires_grad = True

y = torch.tensor(train_df["y"].values, dtype = torch.float64)
y.requires_grad = True
y = y.view(-1, 1)

beta_initial = np.array([0, 0])
beta = torch.zeros(2, 1, dtype = torch.float64)
beta.requires_grad = True

# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    dev = "cuda"
else:
    dev = "cpu"

print(f"Running on: {dev}")

# Define the loss function
def ll(beta):
    xbetahat = torch.mm(x, beta)
    yhat = torch.sigmoid(xbetahat)
    loss = - torch.sum(y * torch.log(yhat) + (1 - y) * torch.log(1 - yhat))
    return loss

# Set up the Adam optimizer
learning_rate = 0.1
optimizer = Adam([beta], lr = learning_rate)

# Define a function to calculate the loss and update model parameters
def calculate_loss():
    optimizer.zero_grad()
    value = ll(beta)
    value.backward()
    return value

num_iter = 100
for i in range(1, num_iter + 1):
  logl = optimizer.step(calculate_loss)
  logl_value = logl.detach().item()
  estimated_weights = beta.detach().numpy()
  if (i + 1) % 10 == 0:
    print(f"Iteration {i + 1} || Log-value {logl_value}")
    print(f"Estimated weights: {estimated_weights}")

estimated_weights_Adam = beta.detach().numpy()
print("Estimated Parameters (Weights):", estimated_weights_Adam)

Running on: cpu
Iteration 10 || Log-value 481.81007249688764
Estimated weights: [[-0.06432935]
 [ 0.82784067]]
Iteration 20 || Log-value 483.2033081110674
Estimated weights: [[-0.17875141]
 [ 1.23860463]]
Iteration 30 || Log-value 480.5549901483747
Estimated weights: [[-0.11729659]
 [ 1.10106756]]
Iteration 40 || Log-value 479.05850454399877
Estimated weights: [[-0.11227977]
 [ 0.8888539 ]]
Iteration 50 || Log-value 479.1051867012501
Estimated weights: [[-0.1309616 ]
 [ 0.90463239]]
Iteration 60 || Log-value 478.8867728450398
Estimated weights: [[-0.13214677]
 [ 0.98764441]]
Iteration 70 || Log-value 478.871395217095
Estimated weights: [[-0.12641927]
 [ 0.97400056]]
Iteration 80 || Log-value 478.8574985568922
Estimated weights: [[-0.12406494]
 [ 0.94452985]]
Iteration 90 || Log-value 478.8473214026619
Estimated weights: [[-0.12461511]
 [ 0.95691198]]
Iteration 100 || Log-value 478.8486356870633
Estimated weights: [[-0.12538916]
 [ 0.96364576]]
Estimated Parameters (Weights): [[-0.12529

In [248]:
x = torch.cat((torch.tensor(train_df[["constant", "X1"]].values, dtype=torch.float64),), dim=1)
x.requires_grad = True

y = torch.tensor(train_df["y"].values, dtype = torch.float64)
y.requires_grad = True
y = y.view(-1, 1)

beta_initial = np.array([0, 0])
beta = torch.zeros(2, 1, dtype = torch.float64)
beta.requires_grad = True

# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    dev = "cuda"
else:
    dev = "cpu"

print(f"Running on: {dev}")

# Define the loss function
def ll(beta):
    xbetahat = torch.mm(x, beta)
    yhat = torch.sigmoid(xbetahat)
    loss = - torch.sum(y * torch.log(yhat) + (1 - y) * torch.log(1 - yhat))
    return loss

# Set up the Adam optimizer
learning_rate = 0.12
optimizer = LBFGS([beta], lr = learning_rate)

# Define a function to calculate the loss and update model parameters
def calculate_loss():
    optimizer.zero_grad()
    value = ll(beta)
    value.backward()
    return value

num_iter = 100
for i in range(1, num_iter + 1):
  logl = optimizer.step(calculate_loss)
  logl_value = logl.detach().item()
  estimated_weights = beta.detach().numpy()
  if (i + 1) % 10 == 0:
    print(f"Iteration {i + 1} || Log-value {logl_value}")
    print(f"Estimated weights: {estimated_weights}")

estimated_weights_LBFGS = beta.detach().numpy()
print("Estimated Parameters (Weights):", estimated_weights_LBFGS)

Running on: cpu
Iteration 10 || Log-value 478.846576313701
Estimated weights: [[-0.12596023]
 [ 0.95827163]]
Iteration 20 || Log-value 478.8465763126951
Estimated weights: [[-0.12596046]
 [ 0.95827329]]
Iteration 30 || Log-value 478.8465763126951
Estimated weights: [[-0.12596046]
 [ 0.95827329]]
Iteration 40 || Log-value 478.8465763126951
Estimated weights: [[-0.12596046]
 [ 0.95827329]]
Iteration 50 || Log-value 478.8465763126951
Estimated weights: [[-0.12596046]
 [ 0.95827329]]
Iteration 60 || Log-value 478.8465763126951
Estimated weights: [[-0.12596046]
 [ 0.95827329]]
Iteration 70 || Log-value 478.8465763126951
Estimated weights: [[-0.12596046]
 [ 0.95827329]]
Iteration 80 || Log-value 478.8465763126951
Estimated weights: [[-0.12596046]
 [ 0.95827329]]
Iteration 90 || Log-value 478.8465763126951
Estimated weights: [[-0.12596046]
 [ 0.95827329]]
Iteration 100 || Log-value 478.8465763126951
Estimated weights: [[-0.12596046]
 [ 0.95827329]]
Estimated Parameters (Weights): [[-0.1259604

In [249]:
# Define the initial values
debugstart = torch.tensor([1, 2, 3], dtype = torch.float64, requires_grad = True)
x0 = torch.tensor([1, 1/2, 1/3], dtype = torch.float64, requires_grad = True)
y1 = torch.tensor([1], dtype = torch.float64, requires_grad = True)

# Calculate xbhat
xbhat = torch.mm(x0.unsqueeze(0), debugstart.unsqueeze(1))
xbhat.retain_grad()

# Calculate yhat
yhat = torch.sigmoid(xbhat)
yhat.retain_grad()

# Calculate the loss
loss = -torch.sum(y1 * torch.log(yhat) + (1 - y1) * torch.log(1 - yhat))
loss.backward()

# Print the tensors and their gradients
print("y:", y1)
print("x:", x0)
print("xbhat:", xbhat)
print("Gradients of xbhat (dloss/dp):", xbhat.grad)
print("yhat:", yhat)
print("Gradients of yhat (dloss/dyhat):", yhat.grad)
print("Loss:", loss.item())

y: tensor([1.], dtype=torch.float64, requires_grad=True)
x: tensor([1.0000, 0.5000, 0.3333], dtype=torch.float64, requires_grad=True)
xbhat: tensor([[3.]], dtype=torch.float64, grad_fn=<MmBackward0>)
Gradients of xbhat (dloss/dp): tensor([[-0.0474]], dtype=torch.float64)
yhat: tensor([[0.9526]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
Gradients of yhat (dloss/dyhat): tensor([[-1.0498]], dtype=torch.float64)
Loss: 0.04858735157374191


NEURAL NETWORK LOGIT

In [200]:
import torch.nn as nn
import torch.nn.functional as F

In [250]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple logistic regression model
class LogitModel(nn.Module):
    def __init__(self, input_dim):
        super(LogitModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)  # Single output unit

    def forward(self, x):
        x = x.to(self.linear.weight.dtype)  # Cast input to the same data type as the model's parameters
        x = self.linear(x)
        x = torch.sigmoid(x)  # Apply sigmoid activation for binary classification
        return x

x = torch.tensor(x_train, dtype = torch.float64)
x.requires_grad = True

y = torch.tensor(y_train, dtype = torch.float64)
y.requires_grad = True
y = y.reshape(-1, 1)

beta = torch.randn(x.shape[1], 1, dtype = torch.float64)
beta.requires_grad = True

input_dim = x.shape[1]

model = LogitModel(input_dim)

loss_function = nn.BCELoss()

optimizer = optim.Adam(model.parameters(), lr = 0.1)

epochs = 5000

# Training loop
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    optimizer.zero_grad()  # Zero the gradients

    # Forward pass
    outputs = model(x)

    outputs = outputs.float()
    y = y.float()

    # Calculate the loss
    loss = loss_function(outputs, y)

    # Backpropagation
    loss.backward()

    # Update the model's parameters
    optimizer.step()

    # Print training progress
    if epoch % (epochs / 10) == 0:
        print(f"Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}")

# View the learned parameters
print("Learned parameters:")
print(f"Model weights: {model.linear.weight[0][0]}, {model.linear.weight[0][1]}") # these are the parameters of interest
print(f"Model bias:{model.linear.bias[0]}")

Epoch [0/5000], Loss: 0.6244
Epoch [500/5000], Loss: 0.5986
Epoch [1000/5000], Loss: 0.5986
Epoch [1500/5000], Loss: 0.5986
Epoch [2000/5000], Loss: 0.5986
Epoch [2500/5000], Loss: 0.5986
Epoch [3000/5000], Loss: 0.5986
Epoch [3500/5000], Loss: 0.5986
Epoch [4000/5000], Loss: 0.5986
Epoch [4500/5000], Loss: 0.5986
Learned parameters:
Model weights: -0.13117192685604095, 0.9582542777061462
Model bias:0.005179565399885178


WORKING PYTORCH LOGISTIC REGRESSION

In [251]:
import torch

x = torch.tensor(x_train, dtype = torch.float64)
x.requires_grad = True

y = torch.tensor(y_train, dtype = torch.float64)
y.requires_grad = True
y = y.reshape(-1, 1)

beta = torch.randn(x.shape[1], 1, dtype = torch.float64)
beta.requires_grad = True

# Initialize parameters
learning_rate = 0.01
num_iterations = 200

# Sigmoid function
def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

# Training loop
for i in range(num_iterations):
    xbetahat = torch.matmul(x, beta)
    yhat = sigmoid(xbetahat)

    loss = -torch.sum(y * torch.log(yhat) + (1 - y) * torch.log(1 - yhat))
    loss.backward()

    with torch.no_grad():
        beta -= learning_rate * beta.grad
    beta.grad.zero_()

   # if (i + 1) % 10 == 0:
    print(f'Iteration {i+1}, Loss: {loss.item()}')

# Get the learned coefficients
learned_beta = beta.detach().numpy()

print(f'Learned Logistic Regression Coefficients (Beta): {learned_beta}')

Iteration 1, Loss: 665.5518600444576
Iteration 2, Loss: 564.8306627212372
Iteration 3, Loss: 486.8035786649813
Iteration 4, Loss: 479.28242033072235
Iteration 5, Loss: 479.026635101907
Iteration 6, Loss: 478.92136907542465
Iteration 7, Loss: 478.8783753497714
Iteration 8, Loss: 478.8600621132225
Iteration 9, Loss: 478.8523052518459
Iteration 10, Loss: 478.8490088337579
Iteration 11, Loss: 478.84760952881254
Iteration 12, Loss: 478.84701509608294
Iteration 13, Loss: 478.84676267710955
Iteration 14, Loss: 478.84665546126496
Iteration 15, Loss: 478.8466099284955
Iteration 16, Loss: 478.84659058933977
Iteration 17, Loss: 478.84658237599206
Iteration 18, Loss: 478.8465788876164
Iteration 19, Loss: 478.84657740607713
Iteration 20, Loss: 478.8465767768439
Iteration 21, Loss: 478.8465765096019
Iteration 22, Loss: 478.84657639610055
Iteration 23, Loss: 478.84657634789517
Iteration 24, Loss: 478.8465763274217
Iteration 25, Loss: 478.84657631872636
Iteration 26, Loss: 478.8465763150333
Iteration 